In [1]:
import pandas
import pickle
import os

In [5]:
pfile = '../ppi_ml/data/features/featmat_allexps_p3c2.pkl'
pfile2 = 'featmat_allexps_p3c2.pkl'

In [11]:
path = os.path.realpath(pfile2)
path

'/stor/work/Marcotte/project/rmcox/leca/notebooks/featmat_allexps_p3c2.pkl'

In [12]:
path, filename = os.path.split(os.path.realpath(pfile2))
print(path,'\n',filename)

/stor/work/Marcotte/project/rmcox/leca/notebooks 
 featmat_allexps_p3c2.pkl


In [8]:
path==''

True

In [17]:
from datetime import datetime as dt
print(dt.now)

<built-in method now of type object at 0x7f3c210f4a60>


In [3]:
with open(pfile, 'rb') as handle:
    df = pickle.load(handle)

In [4]:
df

,ID,arath.iex_1.150p.braycurtis.feat,arath.iex_1.150p.euclidean.feat,arath.iex_1.150p.pearsonR.feat,arath.iex_1.150p.spearmanR.feat,arath.iex_2.150p.braycurtis.feat,arath.iex_2.150p.euclidean.feat,arath.iex_2.150p.pearsonR.feat,arath.iex_2.150p.spearmanR.feat,arath.iex_3.150p.braycurtis.feat,...,xenla.sucrose_3.150p.pearsonR.feat,xenla.sucrose_3.150p.spearmanR.feat,xenla.sucrose_4.150p.braycurtis.feat,xenla.sucrose_4.150p.euclidean.feat,xenla.sucrose_4.150p.pearsonR.feat,xenla.sucrose_4.150p.spearmanR.feat,yeast.iex_1.150p.braycurtis.feat,yeast.iex_1.150p.euclidean.feat,yeast.iex_1.150p.pearsonR.feat,yeast.iex_1.150p.spearmanR.feat
0,ENOG502QPIC ENOG502QR7H,0.980388,5.157008,-0.075497,-0.066312,0.971325,9.577124,-0.067190,-0.015749,0.997534,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ENOG502QPIC KOG0253,0.997407,5.492013,-0.046335,-0.059675,NaN,NaN,NaN,NaN,0.855288,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ENOG502QPIC KOG0734,0.865005,5.916432,0.103333,0.095018,0.998996,11.567026,-0.139190,-0.171415,0.636175,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ENOG502QPIC KOG0739,0.930706,5.596031,0.075926,0.071345,0.899831,9.242962,0.084084,0.115910,0.879471,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ENOG502QPIC KOG0935,0.996862,4.694391,-0.036563,-0.041946,NaN,NaN,NaN,NaN,0.997371,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4491713,ENOG502RYI9 KOG3271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.591567,1.930056,0.797882,0.696994
4491714,ENOG502RYI9 KOG3274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.647338,3.135001,0.531502,0.456474
4491715,ENOG502RYI9 KOG3394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.715621,3.625529,0.439617,0.354524
4491716,ENOG502RYI9 KOG3855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.612288,2.614478,0.523113,0.519076


In [5]:
gs_file = '../ppi_ml/data/gold_stds/all.gold.cmplx.noRibos.merged.txt'

In [12]:
import argparse
import pandas as pd
import pickle
import numpy as np
import re
import random
import time
import datetime as dt
from itertools import combinations
from itertools import accumulate
from collections import defaultdict
def make_pos_dict(gs_file):
    print(f'[{dt.datetime.now()}] Generating grouped positive PPI labels from gold standard complexes ...')
    pos_ppi_dict = dict()
    group_no = 1
    dupes = []
    with open(gs_file, 'r') as f:
        ppis = f.read().splitlines() 
        for p in ppis:
            ogs = p.split(' ')
            fsets = [frozenset({i, j}) for i,j in list(combinations(ogs, 2))]
            pos_ppi_dict.update({group_no: fsets})
            group_no += 1
    print(f'[{dt.datetime.now()}] Finished generating positive PPI labels!')
    return(pos_ppi_dict)

In [13]:
pos_dict = make_pos_dict(gs_file)

[2023-02-08 12:49:13.614944] Generating grouped positive PPI labels from gold standard complexes ...
[2023-02-08 12:49:13.695424] Finished generating positive PPI labels!


In [19]:
pos_dict.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 22

In [16]:
list(pos_dict.values())

[[frozenset({'KOG3599', 'KOG4203'})],
 [frozenset({'KOG0101', 'KOG0730'}),
  frozenset({'KOG0676', 'KOG0730'}),
  frozenset({'KOG0730', 'KOG0985'}),
  frozenset({'KOG0101', 'KOG0676'}),
  frozenset({'KOG0101', 'KOG0985'}),
  frozenset({'KOG0676', 'KOG0985'})],
 [frozenset({'ENOG502R17J', 'KOG0374'})],
 [frozenset({'KOG0439', 'KOG1737'})],
 [frozenset({'KOG3285', 'KOG4593'})],
 [frozenset({'KOG1724', 'KOG3701'}),
  frozenset({'KOG1724', 'KOG2166'}),
  frozenset({'KOG2166', 'KOG3701'})],
 [frozenset({'KOG4031', 'KOG4728'})],
 [frozenset({'KOG0266', 'KOG1273'})],
 [frozenset({'ENOG502QV4A', 'KOG0904'})],
 [frozenset({'KOG0660', 'KOG0941'})],
 [frozenset({'KOG0841', 'KOG2294'})],
 [frozenset({'KOG0594', 'KOG1675'})],
 [frozenset({'ENOG502QT9V', 'KOG1343'}),
  frozenset({'ENOG502QU6B', 'KOG1343'}),
  frozenset({'ENOG502QT9V', 'ENOG502QU6B'})],
 [frozenset({'KOG1721', 'KOG2350'}),
  frozenset({'KOG1079', 'KOG1721'}),
  frozenset({'KOG1079', 'KOG2350'})],
 [frozenset({'ENOG502R17J', 'KOG0379'

In [17]:
all_pos_pairs = [item for sublist in list(pos_dict.values()) for item in sublist]

In [18]:
all_pos_pairs

[frozenset({'KOG3599', 'KOG4203'}),
 frozenset({'KOG0101', 'KOG0730'}),
 frozenset({'KOG0676', 'KOG0730'}),
 frozenset({'KOG0730', 'KOG0985'}),
 frozenset({'KOG0101', 'KOG0676'}),
 frozenset({'KOG0101', 'KOG0985'}),
 frozenset({'KOG0676', 'KOG0985'}),
 frozenset({'ENOG502R17J', 'KOG0374'}),
 frozenset({'KOG0439', 'KOG1737'}),
 frozenset({'KOG3285', 'KOG4593'}),
 frozenset({'KOG1724', 'KOG3701'}),
 frozenset({'KOG1724', 'KOG2166'}),
 frozenset({'KOG2166', 'KOG3701'}),
 frozenset({'KOG4031', 'KOG4728'}),
 frozenset({'KOG0266', 'KOG1273'}),
 frozenset({'ENOG502QV4A', 'KOG0904'}),
 frozenset({'KOG0660', 'KOG0941'}),
 frozenset({'KOG0841', 'KOG2294'}),
 frozenset({'KOG0594', 'KOG1675'}),
 frozenset({'ENOG502QT9V', 'KOG1343'}),
 frozenset({'ENOG502QU6B', 'KOG1343'}),
 frozenset({'ENOG502QT9V', 'ENOG502QU6B'}),
 frozenset({'KOG1721', 'KOG2350'}),
 frozenset({'KOG1079', 'KOG1721'}),
 frozenset({'KOG1079', 'KOG2350'}),
 frozenset({'ENOG502R17J', 'KOG0379'}),
 frozenset({'KOG1724', 'KOG3953'}),
